In [13]:
import os
import polars as pl
import numpy as np
import plotly.express as px

from datetime import datetime, timezone

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

In [14]:
df_p_dwd = pl.read_parquet(os.path.join(DATA_DIRECTORY, "processed", "picarro", "Calibrated_Raw_DWD_Picarro_G2301_413.parquet"))

In [15]:
def process_calibration_time(df, start_date, end_date):
    df_filtered = df.filter(pl.col("datetime").is_between(start_date, end_date))
    
    fig = px.line(df_filtered, x="datetime", y="picarro_corrected")
    fig.show()
    
    fig = px.line(df_filtered, x="datetime", y="h2o_reported")
    fig.show()
    
    data = df_filtered.select(pl.col("picarro_corrected")).to_series().to_list()
        
    return np.median(data[int(len(data)*0.3):int(len(data)*0.95)])

In [16]:
# 81 14:28 - 14:53

start_date = datetime(2024, 12, 3, 14, 28,0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 3, 14, 53, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

524.3138354649163

In [17]:
# 82 14:58 - 15:18

start_date = datetime(2024, 12, 3, 14, 58, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 3, 15, 18, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

523.9366407695632

In [ ]:
# 83 15:25 - 15:45
# Reading: 518.9

start_date = datetime(2024, 12, 3, 15, 25, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 3, 15, 45, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)

/Users/patrickaigner/Documents/PROJECTS/acropolis-visualisation/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/Users/patrickaigner/Documents/PROJECTS/acropolis-visualisation/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in scalar divide



nan

In [ ]:
# 84  16:05 - 16:25
# Reading: 516.0

start_date = datetime(2024, 12, 3, 16, 5, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 12, 3, 16, 25, 0).replace(tzinfo=timezone.utc)

process_calibration_time(df_p_dwd, start_date, end_date)